<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Checking-for-null-values" data-toc-modified-id="Checking-for-null-values-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Checking for null values</a></span></li></ul></li><li><span><a href="#Master-table-creation" data-toc-modified-id="Master-table-creation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Master table creation</a></span></li><li><span><a href="#Users" data-toc-modified-id="Users-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Users</a></span></li><li><span><a href="#Devices" data-toc-modified-id="Devices-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Devices</a></span><ul class="toc-item"><li><span><a href="#Merging-the-devices-data-with-users" data-toc-modified-id="Merging-the-devices-data-with-users-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Merging the devices data with users</a></span></li><li><span><a href="#Notifications-dataset" data-toc-modified-id="Notifications-dataset-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Notifications dataset</a></span><ul class="toc-item"><li><span><a href="#Was-a-notification-sent-in-the-last-x-months?" data-toc-modified-id="Was-a-notification-sent-in-the-last-x-months?-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Was a notification sent in the last x months?</a></span></li><li><span><a href="#Number-of-channels-of-communication" data-toc-modified-id="Number-of-channels-of-communication-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Number of channels of communication</a></span></li><li><span><a href="#Number-of-reasons-for-communication" data-toc-modified-id="Number-of-reasons-for-communication-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Number of reasons for communication</a></span></li><li><span><a href="#Final-columns-to-consider-from-notifications" data-toc-modified-id="Final-columns-to-consider-from-notifications-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Final columns to consider from notifications</a></span></li><li><span><a href="#Merging-users-and-notifications" data-toc-modified-id="Merging-users-and-notifications-3.2.5"><span class="toc-item-num">3.2.5&nbsp;&nbsp;</span>Merging users and notifications</a></span></li></ul></li><li><span><a href="#Transactions" data-toc-modified-id="Transactions-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Transactions</a></span><ul class="toc-item"><li><span><a href="#Transactions-day-of-week-counts" data-toc-modified-id="Transactions-day-of-week-counts-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Transactions day of week counts</a></span></li><li><span><a href="#Number-of-inbound-and-outbound-transactions" data-toc-modified-id="Number-of-inbound-and-outbound-transactions-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Number of inbound and outbound transactions</a></span></li><li><span><a href="#Transaction-state-counts" data-toc-modified-id="Transaction-state-counts-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Transaction state counts</a></span></li><li><span><a href="#Is-user-engaged?" data-toc-modified-id="Is-user-engaged?-3.3.4"><span class="toc-item-num">3.3.4&nbsp;&nbsp;</span>Is user engaged?</a></span></li><li><span><a href="#Amount" data-toc-modified-id="Amount-3.3.5"><span class="toc-item-num">3.3.5&nbsp;&nbsp;</span>Amount</a></span></li><li><span><a href="#Selecting-columns-from-transactions-table-for-model-building" data-toc-modified-id="Selecting-columns-from-transactions-table-for-model-building-3.3.6"><span class="toc-item-num">3.3.6&nbsp;&nbsp;</span>Selecting columns from transactions table for model building</a></span></li><li><span><a href="#Writing-the-flat-table-to-disk" data-toc-modified-id="Writing-the-flat-table-to-disk-3.3.7"><span class="toc-item-num">3.3.7&nbsp;&nbsp;</span>Writing the flat table to disk</a></span></li></ul></li></ul></li><li><span><a href="#Master-table" data-toc-modified-id="Master-table-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Master table</a></span></li></ul></div>

# Define a target metric to measure user engagement. How would you define an engaged vs. unengaged user?


I am assuming that a user can be tagged as engaged, if
    - user has at least one COMPLETED transaction on the last 3 months
and not engaged otherwise 
Note : 
    1. We can tag the users who joined in the last week as engaged user but I want to keep this simple for now
    2. We can tag the users to whom notifications were not delivered in the past 3 months as un engaged user as well ( will do this if time permits)

__NOTE__
1. Using the last date in the transactions table as `today`

In [123]:
import sys
import os
import pandas as pd
from datetime import datetime
import numpy as np
import plotly
import plotly.offline as pyoff
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
init_notebook_mode(connected = True)

The following code is from my github page https://github.com/SCK22/HelperFunctions/blob/master/Plotting/HelperFunctionsPlotlyPlotting.py

In [124]:
"""Helper Functions for Plotting"""

def generate_layout_bar(col_name):
    """
    Generate a layout object for bar chart
    """
    layout_bar = go.Layout(
        autosize=False,  # auto size the graph? use False if you are specifying the height and width
        width=800,  # height of the figure in pixels
        height=600,  # height of the figure in pixels
        title="Distribution of {} column".format(col_name),  # title of the figure
        # more granular control on the title font
        titlefont=dict(
            family='Courier New, monospace',  # font family
            size=14,  # size of the font
            color='black'  # color of the font
        ),
        # granular control on the axes objects
        xaxis=dict(
            tickfont=dict(
                family='Courier New, monospace',  # font family
                size=14,  # size of ticks displayed on the x axis
                color='black'  # color of the font
            )
        ),
        yaxis=dict(
            #         range=[0,100],
            title='Percentage',
            titlefont=dict(
                size=14,
                color='black'
            ),
            tickfont=dict(
                family='Courier New, monospace',  # font family
                size=14,  # size of ticks displayed on the y axis
                color='black'  # color of the font
            )
        ),
        font=dict(
            family='Courier New, monospace',  # font family
            color="white",  # color of the font
            size=12  # size of the font displayed on the bar
        )
    )
    return layout_bar


def plot_count_bar(dataframe_name, col_name, top_n=None):
    """
    Plot a bar chart for the categorical columns
    Arguments:
    dataframe name
    categorical column name
    Output:
    Plot
    """
    # create a table with value counts
    temp = dataframe_name[col_name].value_counts()
    if top_n is not None:
        temp = temp.head(top_n)
    # creating a Bar chart object of plotly
    data = [go.Bar(
            x=temp.index.astype(str),  # x axis values
            y=np.round(temp.values.astype(float) / temp.values.sum(), 4) * 100,  # y axis values
            text=['{}%'.format(i) for i in np.round(temp.values.astype(float) / temp.values.sum(), 4) * 100],
            # text to be displayed on the bar, we are doing this to display the '%' symbol along with the number on the bar
            textposition='auto',  # specify at which position on the bar the text should appear
            marker=dict(color='#0047AB'),)]  # change color of the bar
    # color used here Cobalt Blue
    layout_bar = generate_layout_bar(col_name=col_name)
    fig = go.Figure(data=data, layout=layout_bar)
    return iplot(fig)

def plot_bar(dataframe_name, cat_col_name, num_col_name, top_n = 20):
    """
    Plot a bar chart with the mentioned columns
    Arguments:
    dataframe name
    categorical column name
    numeric column name
    Output:
    Plot
    """
    # create a table with value counts
    dataframe_name = dataframe_name.sort_values(by = num_col_name, ascending = False)
    dataframe_name = dataframe_name.head(top_n)
    x = dataframe_name[cat_col_name]
    y = dataframe_name[num_col_name]
    # creating a Bar chart object of plotly
    data = [go.Bar(
            x=x,  # x axis values
            y=y,  # y axis values
            text=['{}%'.format(np.round(i,2)) for i in y],
            # text to be displayed on the bar, we are doing this to display the '%' symbol along with the number on the bar
            textposition='auto',  # specify at which position on the bar the text should appear
            marker=dict(color='#0047AB'),)]  # change color of the bar
    # color used here Cobalt Blue
    layout_bar = generate_layout_bar(col_name=cat_col_name)
    fig = go.Figure(data=data, layout=layout_bar)
    return iplot(fig)


def plot_hist(dataframe, col_name):
    """Plot histogram"""
    data = [go.Histogram(x=dataframe[col_name],
                         marker=dict(
        color='#CC0E1D',  # Lava (#CC0E1D)
        #         color = 'rgb(200,0,0)'   # you can provide color in HEX format or rgb format, genrally programmers prefer HEX format as it is a single string value and easy to pass as a variable
    ))]
    layout = go.Layout(title="Histogram of {}".format(col_name))
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig)


def plot_multi_box(dataframe, col_name, num_col_name):
    """Plot multiple box plots based on the levels in a column"""
    data = []
    for i in dataframe[col_name].unique():
        trace = go.Box(y=dataframe[num_col_name][dataframe[col_name] == i],
                       name=i)
        data.append(trace)
    layout = go.Layout(title="Boxplot of levels in {} for {} column".format(col_name, num_col_name))
    fig = go.Figure(data=data, layout=layout)
    return (iplot(fig))

In [125]:
data_path = "data/"

In [126]:
def load_data(file_name):
    return pd.read_csv(data_path+f"{file_name}"+".csv", na_values=[" ", "", np.nan, "nan", "Na"])

In [127]:
os.listdir(data_path)

['notifications.csv',
 'notifications_processed.csv',
 'rev-devices.csv',
 'rev-notifications.csv',
 'rev-transactions.csv',
 'rev-users.csv',
 'transactions_processed.csv',
 'users.csv',
 'users_processed.csv']

In [128]:
def convert_to_pandas_datetime(col):
    return pd.to_datetime(col)

In [129]:
def check_null(df):
    return df.isnull().sum()

Converting dates to pandas datetime format

In [130]:
devices = load_data("rev-devices")
notifications = load_data("notifications_processed")
transactions = load_data("transactions_processed")
users = load_data("users_processed")

In [131]:
users.created_date = convert_to_pandas_datetime(users.created_date)

transactions.created_date = convert_to_pandas_datetime(transactions.created_date)

notifications.created_date = convert_to_pandas_datetime(notifications.created_date)

In [132]:
today = transactions.created_date.max()

#### Checking for null values

In [133]:
check_null(users)

user_id                                        0
birth_year                                     0
country                                        0
city                                           0
created_date                                   0
user_settings_crypto_unlocked                  0
plan                                           0
attributes_notifications_marketing_push     6610
attributes_notifications_marketing_email    6610
num_contacts                                   0
num_referrals                                  0
num_successful_referrals                       0
age                                            0
dtype: int64

In [134]:
check_null(notifications)

reason          0
channel         0
status          0
user_id         0
created_date    0
dtype: int64

In [135]:
check_null(transactions)

transaction_id                 0
transactions_type              0
transactions_currency          0
amount_usd                     0
transactions_state             0
ea_cardholderpresence          0
ea_merchant_mcc          1158658
ea_merchant_city         1158803
ea_merchant_country      1158979
direction                      0
user_id                        0
created_date                   0
dayofweek                      0
transactionmonth               0
dtype: int64

In [136]:
check_null(devices)

brand      0
user_id    0
dtype: int64

> There are null values in some of the columns in the dataset, Instead of spending time on imputation, I will not use these columns are most of these columns for analysis

In [137]:
null_coulmns = []
for i in [users,transactions,notifications,devices]:
        null_coulmns.extend(i.columns[i.isnull().sum()>0])

In [138]:
null_coulmns

['attributes_notifications_marketing_push',
 'attributes_notifications_marketing_email',
 'ea_merchant_mcc',
 'ea_merchant_city',
 'ea_merchant_country']

### Master table creation

- First , the task of creating a master table is at hand
    - The common column is the user_id
    - There are multiple transactions per user in the transactions table, aggregating is the first task for this dataset
    - There are multiple notifications per user in the notifications table, aggregating is the first task for this dataset
    - devices can be used as is to join (with the pre processing done in the EDA.ipynb)


### Users

In [139]:
users.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'age'],
      dtype='object')

In [140]:
users_plan = pd.crosstab(users.user_id, users.plan).reset_index()

In [141]:
users_plan.columns

Index(['user_id', 'METAL', 'METAL_FREE', 'PREMIUM', 'PREMIUM_FREE',
       'PREMIUM_OFFER', 'STANDARD'],
      dtype='object', name='plan')

In [142]:
users = pd.merge(users,users_plan ,left_on="user_id", right_on = "user_id")

### Devices

In [143]:
user_device_brand = pd.crosstab(devices.user_id,devices.brand).reset_index()

In [144]:
user_device_brand.columns

Index(['user_id', 'Android', 'Apple', 'Unknown'], dtype='object', name='brand')

In [145]:
devices = user_device_brand.loc[:,['user_id', 'Android', 'Apple',]]

#### Merging the devices data with users

In [146]:
users = pd.merge(users, devices,left_on="user_id", right_on="user_id")

In [147]:
users.to_csv("model_data/users.csv")

#### Notifications dataset

In [148]:
num_notifications_user = notifications.groupby(["user_id"]).size().to_frame().reset_index()

In [149]:
num_notifications_user.columns = ["user_id", "num_notifications"]

In [150]:
num_notifications_user.head()

,user_id,num_notifications
0,user_0,8
1,user_1,7
2,user_10,6
3,user_100,4
4,user_1000,18


In [151]:
notifications = pd.merge(notifications, num_notifications_user,left_on="user_id", right_on="user_id")

##### Was a notification sent in the last x months?

I will try 15,30,45,60,90,180 for x values

- Determine if the user was notified in the las x days
- Calculate the number of times he was notified
- Merge the dataset with the original

In [152]:
x = [15,30,45,60,90,180]
for d in x:
    notifications["notified_{}".format(d)] = notifications.created_date.apply(lambda x : (today - x).days>d) # find if the user was notified
    temp = notifications.groupby(["user_id"]).agg({"notified_{}".format(d):np.sum}).reset_index() # calcuate the number of times he was notified
    temp.columns = ["user_id", "num_notified_{}".format(d)] # renaming the columns
    notifications = pd.merge(notifications, temp, left_on="user_id", right_on= "user_id") # merging the two datasets

In [153]:
notifications.columns

Index(['reason', 'channel', 'status', 'user_id', 'created_date',
       'num_notifications', 'notified_15', 'num_notified_15', 'notified_30',
       'num_notified_30', 'notified_45', 'num_notified_45', 'notified_60',
       'num_notified_60', 'notified_90', 'num_notified_90', 'notified_180',
       'num_notified_180'],
      dtype='object')

##### Number of channels of communication

In [154]:
num_channles_of_com = notifications.groupby("user_id").agg({"channel" : lambda x : len(np.unique(x))}).reset_index()
num_channles_of_com.columns = ["user_id", "num_channles_of_com"]

In [155]:
num_channles_of_com.head()

,user_id,num_channles_of_com
0,user_0,2
1,user_1,2
2,user_10,2
3,user_100,2
4,user_1000,3


In [156]:
notifications = pd.merge(notifications, num_channles_of_com,left_on= "user_id", right_on = "user_id")

In [157]:
notifications.columns

Index(['reason', 'channel', 'status', 'user_id', 'created_date',
       'num_notifications', 'notified_15', 'num_notified_15', 'notified_30',
       'num_notified_30', 'notified_45', 'num_notified_45', 'notified_60',
       'num_notified_60', 'notified_90', 'num_notified_90', 'notified_180',
       'num_notified_180', 'num_channles_of_com'],
      dtype='object')

##### Number of reasons for communication

In [158]:
num_reasons_of_com = notifications.groupby("user_id").agg({"reason" : lambda x : len(np.unique(x))}).reset_index()
num_reasons_of_com.columns = ["user_id", "num_reasons_com"]
notifications = pd.merge(notifications, num_reasons_of_com,left_on= "user_id", right_on = "user_id")

##### Final columns to consider from notifications

In [159]:
notifications.columns

Index(['reason', 'channel', 'status', 'user_id', 'created_date',
       'num_notifications', 'notified_15', 'num_notified_15', 'notified_30',
       'num_notified_30', 'notified_45', 'num_notified_45', 'notified_60',
       'num_notified_60', 'notified_90', 'num_notified_90', 'notified_180',
       'num_notified_180', 'num_channles_of_com', 'num_reasons_com'],
      dtype='object')

In [160]:
notifications_cols = ["user_id",'num_notifications',
 'num_notified_15',
 'num_notified_30',
 'num_notified_45',
 'num_notified_60',
 'num_notified_90',
 'num_notified_180',
 'num_channles_of_com',
 'num_reasons_com']

In [161]:
notifications_flat = notifications.loc[:, notifications_cols]

In [162]:
notifications_flat = notifications_flat.drop_duplicates("user_id")

In [163]:
notifications_flat.shape

(18953, 10)

In [164]:
notifications_flat.head()

,user_id,num_notifications,num_notified_15,num_notified_30,num_notified_45,num_notified_60,num_notified_90,num_notified_180,num_channles_of_com,num_reasons_com
0,user_7086,10,10.0,10.0,10.0,10.0,10.0,0.0,2,2
10,user_6598,11,11.0,11.0,10.0,10.0,10.0,3.0,2,6
21,user_4151,7,6.0,6.0,6.0,6.0,6.0,1.0,2,3
28,user_1408,4,4.0,4.0,4.0,4.0,3.0,1.0,1,2
32,user_6292,10,10.0,10.0,10.0,10.0,8.0,3.0,2,4


In [165]:
notifications_flat.to_csv("model_data/notifications_flat.csv",index=False)

##### Merging users and notifications

In [166]:
users.shape

(19430, 21)

In [167]:
users.user_id.nunique()

19430

In [168]:
master = pd.merge(users, notifications_flat, left_on= "user_id", right_on= "user_id", how = "inner")

In [169]:
master.to_csv("model_data/master.csv",index= False)

#### Transactions

##### Transactions day of week counts
Count the numbe or transactions a user has done on a week day

In [170]:
weekday_transactions = pd.crosstab(transactions.user_id, transactions.dayofweek).reset_index()
["num_transactions_{}".format(i) for i in weekday_transactions.columns[1:]]

['num_transactions_Fri',
 'num_transactions_Mon',
 'num_transactions_Sat',
 'num_transactions_Sun',
 'num_transactions_Thu',
 'num_transactions_Tue',
 'num_transactions_Wed']

In [171]:
weekday_transactions.columns = ["user_id", 'num_transactions_Fri',
 'num_transactions_Mon',
 'num_transactions_Sat',
 'num_transactions_Sun',
 'num_transactions_Thu',
 'num_transactions_Tue',
 'num_transactions_Wed']

In [172]:
weekday_transactions.head()

,user_id,num_transactions_Fri,num_transactions_Mon,num_transactions_Sat,num_transactions_Sun,num_transactions_Thu,num_transactions_Tue,num_transactions_Wed
0,user_0,66,64,51,54,51,53,50
1,user_1,15,14,26,16,23,14,21
2,user_10,160,143,165,135,130,134,186
3,user_100,13,15,16,24,14,23,28
4,user_1000,1,8,3,3,9,11,4


In [173]:
transactions.columns

Index(['transaction_id', 'transactions_type', 'transactions_currency',
       'amount_usd', 'transactions_state', 'ea_cardholderpresence',
       'ea_merchant_mcc', 'ea_merchant_city', 'ea_merchant_country',
       'direction', 'user_id', 'created_date', 'dayofweek',
       'transactionmonth'],
      dtype='object')

In [174]:
transactions = pd.merge(transactions, weekday_transactions, left_on="user_id", right_on= "user_id")

In [175]:
transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,...,created_date,dayofweek,transactionmonth,num_transactions_Fri,num_transactions_Mon,num_transactions_Sat,num_transactions_Sun,num_transactions_Thu,num_transactions_Tue,num_transactions_Wed
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,2018-04-03 03:34:21.784487,Tue,4,30,59,30,25,31,55,31
1,transaction_378,TRANSFER,AED,14.47,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,2018-04-08 04:43:53.933487,Sun,4,30,59,30,25,31,55,31
2,transaction_379,ATM,AED,41.33,COMPLETED,False,6011.0,Abu Dhabi,ARE,OUTBOUND,...,2018-04-07 18:41:07.029487,Sat,4,30,59,30,25,31,55,31
3,transaction_380,CARD_PAYMENT,AED,6.61,COMPLETED,False,5814.0,Dubai,ARE,OUTBOUND,...,2018-04-04 23:33:57.894487,Wed,4,30,59,30,25,31,55,31
4,transaction_381,TRANSFER,AED,0.21,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,2018-04-04 19:39:31.674487,Wed,4,30,59,30,25,31,55,31


##### Number of inbound and outbound transactions

In [176]:
direction_transactions = pd.crosstab(transactions.user_id, transactions.direction).reset_index()

In [177]:
direction_transactions.columns = ['user_id', 'INBOUND_count', 'OUTBOUND_count']

In [178]:
direction_transactions.head()

,user_id,INBOUND_count,OUTBOUND_count
0,user_0,69,320
1,user_1,32,97
2,user_10,284,769
3,user_100,25,108
4,user_1000,16,23


In [179]:
transactions.columns

Index(['transaction_id', 'transactions_type', 'transactions_currency',
       'amount_usd', 'transactions_state', 'ea_cardholderpresence',
       'ea_merchant_mcc', 'ea_merchant_city', 'ea_merchant_country',
       'direction', 'user_id', 'created_date', 'dayofweek', 'transactionmonth',
       'num_transactions_Fri', 'num_transactions_Mon', 'num_transactions_Sat',
       'num_transactions_Sun', 'num_transactions_Thu', 'num_transactions_Tue',
       'num_transactions_Wed'],
      dtype='object')

In [180]:
transactions = pd.merge(transactions, direction_transactions, left_on="user_id", right_on= "user_id")

##### Transaction state counts


In [181]:
t_state_transactions = pd.crosstab(transactions.user_id, transactions.transactions_state).reset_index()

In [182]:
["{}_count".format(i) for i in t_state_transactions.columns[1:]]

['CANCELLED_count',
 'COMPLETED_count',
 'DECLINED_count',
 'FAILED_count',
 'PENDING_count',
 'REVERTED_count']

In [183]:
t_state_transactions.columns = ['user_id', 'CANCELLED_count',
 'COMPLETED_count',
 'DECLINED_count',
 'FAILED_count',
 'PENDING_count',
 'REVERTED_count']

In [184]:
t_state_transactions.head()

,user_id,CANCELLED_count,COMPLETED_count,DECLINED_count,FAILED_count,PENDING_count,REVERTED_count
0,user_0,0,350,20,4,0,15
1,user_1,1,114,11,0,0,3
2,user_10,0,846,45,99,1,62
3,user_100,0,110,22,1,0,0
4,user_1000,0,35,1,1,0,2


In [185]:
transactions = pd.merge(transactions, t_state_transactions, left_on="user_id", right_on= "user_id")

##### Is user engaged?
As mentioned above, I will consider that a user is engaged if they made a tranasaction in the last 3 months (180 days)

In [186]:
# creating a new column called transaction_age, which is the difference bewteen the `today` variable and the date of transaction
# transactions["transaction_age"] = transactions.created_date.apply(lambda x : (today - x).days)
# pd.crosstab(transactions.user_id, transactions.transaction_age)

In [187]:
is_user_active = transactions.created_date.apply(lambda x : (today - x).days<180)

In [188]:
is_user_active.head()

0    False
1    False
2    False
3    False
4    False
Name: created_date, dtype: bool

In [189]:
is_user_active.value_counts()

True     1700845
False    1039230
Name: created_date, dtype: int64

In [190]:
transactions["is_user_active"] = is_user_active
# pd.merge(transactions, is_user_active, left_on="user_id", right_on= "user_id")

In [191]:
transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,...,num_transactions_Wed,INBOUND_count,OUTBOUND_count,CANCELLED_count,COMPLETED_count,DECLINED_count,FAILED_count,PENDING_count,REVERTED_count,is_user_active
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,31,53,208,0,240,14,4,0,3,False
1,transaction_378,TRANSFER,AED,14.47,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,31,53,208,0,240,14,4,0,3,False
2,transaction_379,ATM,AED,41.33,COMPLETED,False,6011.0,Abu Dhabi,ARE,OUTBOUND,...,31,53,208,0,240,14,4,0,3,False
3,transaction_380,CARD_PAYMENT,AED,6.61,COMPLETED,False,5814.0,Dubai,ARE,OUTBOUND,...,31,53,208,0,240,14,4,0,3,False
4,transaction_381,TRANSFER,AED,0.21,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,31,53,208,0,240,14,4,0,3,False


In [192]:
def is_user_engaged(x):
    if np.sum(x)>=1:
        return "ENGAGED"
    else:
        return "NOT_ENGAGED"

In [193]:
is_user_engaged_df = transactions.groupby("user_id").agg({"is_user_active" : lambda x : is_user_engaged(x)}).reset_index()

In [194]:
is_user_engaged_df.head()

,user_id,is_user_active
0,user_0,ENGAGED
1,user_1,ENGAGED
2,user_10,ENGAGED
3,user_100,ENGAGED
4,user_1000,NOT_ENGAGED


In [195]:
is_user_engaged_df.is_user_active.value_counts()

ENGAGED        16110
NOT_ENGAGED     2656
Name: is_user_active, dtype: int64

In [196]:
plot_count_bar(is_user_engaged_df ,"is_user_active")


__Based on the criteria (at least one transaction in last 180 days) there are 15% of the users who are not engaged__

In [197]:
is_user_engaged_df.columns = ["user_id", "is_user_engaged"]

In [198]:
transactions = pd.merge(transactions, is_user_engaged_df, left_on="user_id", right_on= "user_id")

##### Amount
I am taking the median,mean,min,max and standard deviation of the trnsactions amounts

In [199]:
transaction_amounts = transactions.groupby("user_id").agg({"amount_usd" : [np.median,np.mean,np.min,np.max,np.std]}).reset_index()
transaction_amounts.columns = ['user_id','median_usd', 'mean_usd', 'min_usd', 'max_usd', 'std_usd']

In [200]:
transaction_amounts.head()

,user_id,median_usd,mean_usd,min_usd,max_usd,std_usd
0,user_0,8.43,35.176787,0.00,582.18,66.833649
1,user_1,23.65,47.316899,0.00,350.00,63.369764
2,user_10,12.77,49.525109,0.00,2947.24,173.066372
3,user_100,85.49,156.224211,0.55,709.21,180.201766
4,user_1000,500.00,552.944615,1.00,1000.03,348.723115


In [201]:
transactions = pd.merge(transactions, transaction_amounts, left_on="user_id", right_on= "user_id")

##### Selecting columns from transactions table for model building

In [202]:
transactions.columns

Index(['transaction_id', 'transactions_type', 'transactions_currency',
       'amount_usd', 'transactions_state', 'ea_cardholderpresence',
       'ea_merchant_mcc', 'ea_merchant_city', 'ea_merchant_country',
       'direction', 'user_id', 'created_date', 'dayofweek', 'transactionmonth',
       'num_transactions_Fri', 'num_transactions_Mon', 'num_transactions_Sat',
       'num_transactions_Sun', 'num_transactions_Thu', 'num_transactions_Tue',
       'num_transactions_Wed', 'INBOUND_count', 'OUTBOUND_count',
       'CANCELLED_count', 'COMPLETED_count', 'DECLINED_count', 'FAILED_count',
       'PENDING_count', 'REVERTED_count', 'is_user_active', 'is_user_engaged',
       'median_usd', 'mean_usd', 'min_usd', 'max_usd', 'std_usd'],
      dtype='object')

In [203]:
transactions.head()

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,...,FAILED_count,PENDING_count,REVERTED_count,is_user_active,is_user_engaged,median_usd,mean_usd,min_usd,max_usd,std_usd
0,transaction_0,TRANSFER,AED,4.55,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,4,0,3,False,ENGAGED,10.0,24.931264,0.0,300.0,43.845133
1,transaction_378,TRANSFER,AED,14.47,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,4,0,3,False,ENGAGED,10.0,24.931264,0.0,300.0,43.845133
2,transaction_379,ATM,AED,41.33,COMPLETED,False,6011.0,Abu Dhabi,ARE,OUTBOUND,...,4,0,3,False,ENGAGED,10.0,24.931264,0.0,300.0,43.845133
3,transaction_380,CARD_PAYMENT,AED,6.61,COMPLETED,False,5814.0,Dubai,ARE,OUTBOUND,...,4,0,3,False,ENGAGED,10.0,24.931264,0.0,300.0,43.845133
4,transaction_381,TRANSFER,AED,0.21,COMPLETED,False,NaN,NaN,NaN,OUTBOUND,...,4,0,3,False,ENGAGED,10.0,24.931264,0.0,300.0,43.845133


In [204]:
transactions_cols = ["user_id", 'num_transactions_Fri', 'num_transactions_Mon', 'num_transactions_Sat',
       'num_transactions_Sun', 'num_transactions_Thu', 'num_transactions_Tue',
       'num_transactions_Wed', 'INBOUND_count', 'OUTBOUND_count',
       'CANCELLED_count', 'COMPLETED_count', 'DECLINED_count', 'FAILED_count',
       'PENDING_count', 'REVERTED_count','is_user_engaged','median_usd', 'mean_usd', 'min_usd', 'max_usd', 'std_usd',]

In [205]:
transactions_flat = transactions.loc[:, transactions_cols]

In [206]:
transactions_flat = transactions_flat.drop_duplicates("user_id")

In [207]:
transactions_flat.shape

(18766, 22)

##### Writing the flat table to disk

In [208]:
transactions_flat.to_csv("transactions_flat.csv", index= False)

### Master table

In [209]:
# master =pd.read_csv("model_data/master.csv")

In [210]:
master = pd.merge(master, transactions_flat, left_on="user_id", right_on= "user_id")

In [211]:
master.shape

(18337, 51)

__The number of users who made transactions are 18337, will continue with model building with these users data__

In [212]:
master.to_csv("model_data/master.csv", index= False)